 # Lane Detection
 ##### Cody Weaver 
 ##### Source: https://www.sciencedirect.com/science/article/pii/S0262885603002105?casa_token=EI57d-sffaMAAAAA:wkjUpy9_k3AtEsvXlLD2Gg2wxpcxQfVMsnWSwuiy2q8eU1CKje10EderkPg0H2y1TMUiFHdBDw#FIG7
 
## CHEVP Algorithm
The CHEVP (Canny/Hough Estimation of Vanishing Points) algorithm is capable of detecting lanes and setting control points for paved roads. It uses a B Snake lane model and has 5 steps. 

##### Psuedocode:
    Create edge map using Canny Edge Detector
    Detect straight lines using Hough Transform
    Detect horizon and vanishing point
    Estimate mid line of road and boundries 
    Calculate control points
    
Once edges are detected the image is split horizontally into multiple sections. For each section a Hough Transform is done to look for lines in the edge map. After a Hough Transform has been applied to an image section the result looks something like this: (**this image was not split into sections**)

<p style="text-align: center;"><img src="img/road_output.jpg"/></p> 

Where the blue lines are the "road" lines detected by the Hough Transform. If no lines can be found for a given section of the image you can estimate the road lines using the other sections which could find lines. The vanishing point, which is just where the road "stops" on a given image is calculated by extending each line detected and counting intersections of pairs of lines. Each intersection of two lines votes for a vanishing point. The vanishing points are calculated for each image section seperatly. The vanishing line and horizon of the road is calculated based on the intersections of the road lines. Here is an example of what this might look like on a road from Wang et al.

<p style="text-align: center;"><img src="img/wang_example1.jpg"/></p>

The black bar above the photos represents the horizon. Control points $Q_0, Q_1, Q_2$ are then calculated, another example from Wang et al.

<p style="text-align: center;"><img src="img/wang_example2.jpg"/></p>

## Problems I Encountered 

Canny edge detection works very well for well structured roads with high contrast road markings or road boundries. However, when I applied Canny to the data Trimble gave us the result looked like this: (**White lines represent lines found by Hough Transform**)

<p style="text-align: center;"><img src="img/test_out3.jpg"/></p>

The ground has too much noise that interferes with trying to find edges of plants and rows. There may be a way to eliminate enough noise to detect edges of rows and plants cleanly but I was not able to find such a way. However, I was able to detect some row lines using Canny but only far away from the tractor, 

<p style="text-align: center;"><img src="img/test_out.jpg"/></p>

When I apply Canny and Hough Transform to sections of the image closer to the tractor the result is the following,

<p style="text-align: center;"><img src="img/canny_lines.jpg"/></p>

There was nothing I tried that gave me better results.

I Decided to try a different method, where instead of applying Canny I extracted the green channel of the image and then applied a color mask that detected green channel values between a given range. This gave me the following image:

<p style="text-align: center;"><img src="img/mask.jpg"/></p>

This gives much better information about the rows near to the tractor but at longer ranges the detail is much worse. I then split the image into sections. I used 4 vertical sections and 4 horizonal sections. The horizontal sections are not used in CHEVP but they help as there are both vertical and horizontal rows in the field and only the vertical ones are needed. After splitting the image I applied a Hough Transform to the masked image section to detect lines and this resulted in the following:

<p style="text-align: center;"><img src="img/mask_lines_example1.jpg"/></p>

Which is much cleaner and perfectly detected the rows in the middle of the image, although there were some lines found which are not parallel to the rows.

However, the row lines detected are not as clean in the top half of the image,

<p style="text-align: center;"><img src="img/mask_lines_example2.jpg"/></p>

This could in theory be a huge problem, if we can only detect lines beneath us it could cause a lot of issues.

#### Other Problems

* Didn't have time to implement the full algorithm
* The modified method I used relies on color data, I don't know if color will be reliable to use in this way.
* CHEVP doesn't work because there is too much noise from the ground to cleanly detect edges.
* My method isn't as effective further away from tractor. 
* So many variables that affect the final outcome and I haven't had time to experiment with all of the possibilities.

#### B-Snake lane model
However, the lane model described by Wang et al. seems straight-forward and easy to use/understand. Maybe we can adapt this lane model to whatever our final algo is.

#### Pseudocode for my modified algorithm
    mask image to extract green features
    blur masked image to reduce noise
    split image into sections
    for each section:
        apply hough transform
        #calculate vanishing points
    #draw center line
    #draw control points
    
    //I have only implemented the algorithm up to the hough transform so far

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import copy

In [2]:
################################
# CONST DECLARATIONS
################################
MASK_1 = np.asarray([36, 25, 25])
MASK_2 = np.asarray([86,255,255])
BLUR_KERNEL_SIZE = (7, 7)
V_SECTIONS = 8
H_SECTIONS = 7
MIN_LINE_LENGTH = 1080//15
MAX_LINE_GAP = 1080//54

In [3]:
################################
# HELPER FUNCTIONS
################################
# Reads Image and returns image array
# args: string: path to image, bool: convert to hsv, bool: convert to grayscale 
def read_image(path, gray=False):
    if gray: flag = cv2.IMREAD_GRAYSCALE
    else: flag = cv2.IMREAD_UNCHANGED
    
    return cv2.imread(path, flag)

# Mask image and returns
# args: array: image in hsv format
def mask_image(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return cv2.inRange(hsv_img, MASK_1, MASK_2)

# Blur image and returns
def blur_image(img):
    return cv2.GaussianBlur(img, BLUR_KERNEL_SIZE, sigmaX=0, sigmaY=0)

def get_section(img, section_num):
    rows, cols = np.shape(img)
    section = copy.deepcopy(img)
    if section_num == 1 or section_num == 2:
        row_range = range((section_num-1)*(rows//V_SECTIONS), (section_num)*(rows//V_SECTIONS))
    elif section_num == 3:
        row_range = range((section_num-1)*(rows//V_SECTIONS), (section_num+1)*(rows//V_SECTIONS))
    elif section_num == 4:
        row_range = range(section_num*(rows//V_SECTIONS), (section_num+4)*(rows//V_SECTIONS))
                          
    if section_num == 1:
        col_range = range(3*(cols//H_SECTIONS), 4*(cols//H_SECTIONS))
    elif section_num == 2 or section_num == 3:
        col_range = range(2*(cols//H_SECTIONS), 5*(cols//H_SECTIONS))
    elif section_num == 4:
        col_range = range(1*(cols//H_SECTIONS), 6*(cols//H_SECTIONS))
    
    for i in range(rows):
        for j in range(cols):
            if (i not in row_range) or (j not in col_range):
                section[i][j] = 0
    
    return section    

def get_candidate_lines(lines, rows, cols):
    candidate_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if (1):#(np.abs(x2-x1) < cols//14 and np.abs(y2-y1) > rows//8):
            candidate_lines.append(line)
            
    return candidate_lines       

def draw_lines(img, lines):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 1)
            
def extend_lines(lines, section_num):
    extended_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if (x2-x1) == 0: # finds slope
            m = (y2-y1)/0.01
        else:
            m = (y2-y1)/(x2-x1)
        b = y1
        step = (section_num+1)*(1080//V_SECTIONS)
        if (m > 0): # extends lines towards the top of the photo
            extended_lines.append([[0, int((0-x1)*m+b), x2, y2]])
        else: 
            extended_lines.append([[x1, y1, x2+step, int((x2-x1+step)*m+b)]])
            
    return extended_lines

In [17]:
########################## 
# HOUGH LINE DETECTION
##########################
class Config:
    RHO=1
    THETA=[np.pi/3, np.pi/3, np.pi/5, np.pi/13]
    THRESHOLD=100
    MIN_LINE_LENGTH=[1080//15, 1080//15, 1080//15, 1080//10]
    MAX_LINE_GAP=[0, 1080//54, 1080//27, 1080//20]

def LineDetectionHelper(img, section):
    section = section - 1 
    lines = cv2.HoughLinesP(img, 
                            Config.RHO, 
                            Config.THETA[section], 
                            Config.THRESHOLD, 
                            minLineLength=Config.MIN_LINE_LENGTH[section], 
                            maxLineGap=Config.MAX_LINE_GAP[section]
                           )
    return lines

def LineDetection(img):
    lines = []
    for section_num in range(1, 5):
        section = get_section(img, section_num)
        lines.append(LineDetectionHelper(section, section_num))
    return lines

In [21]:
##############################
# VANISHING POINTS ESTIMATION
##############################

def EstimateVP(img, section_num, lines):
    pass

In [19]:
img = read_image('img/test_img.jpg')

mask = mask_image(img)
blurred = blur_image(mask)

lines = LineDetection(blurred)
    
for lines in lines:
    draw_lines(img, lines)

cv2.imwrite('img/mask_lines_example.jpg', img)

True

#### **Configs:**
**SECTION 1:**
- blur_kernel_size = (7, 7)
- H_SECTIONS = 7
- V_SECTIONS = 8
- col_range = {3,4}
- row_range = {1}
- get_candidate_lines
    - NOT USED
    - append conditon = if (np.abs(x2-x1) < cols//64 and np.abs(y2-y1) > rows//256) 
- extend_lines
    - extend lines math:
            if (m > 0): # extends lines towards the top of the photo
                extended_lines.append([[0, int((0-x1)*m+b), x2, y2]])
            else: 
                extended_lines.append([[x1, y1, x2+50, int((x2-x1+50)*m+b)]])

- HoughLinesP 
    - rho = 1
    - theta = pi/3
    - threshold = 100
    - minLineLength = 1080//15
    - maxLineGap = 0
        
                    
**SECTION 2:**
- blur_kernel_size = (7, 7)
- H_SECTIONS = 7
- V_SECTIONS = 8
- col_range = {2,..,5}
- row_range = {2}
- get_candidate_lines
- NOT USED
- extend_lines
- extend lines math:
        step = section_num*(1080//V_SECTIONS)
        if (m > 0): # extends lines towards the top of the photo
            extended_lines.append([[0, int((0-x1)*m+b), x2, y2]])
        else: 
            extended_lines.append([[x1, y1, x2+step, int((x2-x1+step)*m+b)]])

- HoughLinesP 
    - rho = 1
    - theta = pi/3
    - threshold = 100
    - minLineLength = 1080//15
    - maxLineGap = 1080//54

**SECTION 3:**
- blur_kernel_size = (7, 7)
- H_SECTIONS = 7
- V_SECTIONS = 8
- col_range = {2,..,5}
- row_range = {3,..,4}
- get_candidate_lines
- NOT USED
- extend_lines
- extend lines math:
        step = (section_num+1)*(1080//V_SECTIONS)
        if (m > 0): # extends lines towards the top of the photo
            extended_lines.append([[0, int((0-x1)*m+b), x2, y2]])
        else: 
            extended_lines.append([[x1, y1, x2+step, int((x2-x1+step)*m+b)]])

- HoughLinesP 
    - rho = 1
    - theta = pi/5
    - threshold = 100
    - minLineLength = 1080//15
    - maxLineGap = 1080//27
    
**SECTION 4:**
- blur_kernel_size = (7, 7)
- H_SECTIONS = 7
- V_SECTIONS = 8
- col_range = {1,..,6}
- row_range = {5,..,8}
- get_candidate_lines
- NOT USED
- extend_lines
- extend lines math:
        step = (section_num+1)*(1080//V_SECTIONS)
        if (m > 0): # extends lines towards the top of the photo
            extended_lines.append([[0, int((0-x1)*m+b), x2, y2]])
        else: 
            extended_lines.append([[x1, y1, x2+step, int((x2-x1+step)*m+b)]])

- HoughLinesP 
    - rho = 1
    - theta = pi/13
    - threshold = 100
    - minLineLength = 1080//10
    - maxLineGap = 1080//20